In [16]:
import csv
import os
import re
import sys
import glob
import libertyParser
from liberty.parser import parse_liberty
from iteration_utilities import deepflatten
def flatten(nested):
	for sublist in nested:
		if isinstance(sublist,str):
			return sublist
		else:
			for element in flatten(sublist):
				return element

def RwaDate2Csv( RawDataPath,CsvPath ):
    with open(CsvPath, 'w', newline='') as csvfile:
        
        liberty_file = RawDataPath
        writer = csv.writer(csvfile)
        
        ### columns_names
        COLUMN_NAMES = ['temperature', 'voltage', 'capacitance', 'rise_capacitance', 'fall_capacitance', 
                'cell_rise', 'rise_transition', 'cell_fall', 'fall_transition', 'rise_power', 'fall_power',
                'index1_1', 'index1_2', 'index1_3', 'index1_4', 'index1_5', 'index1_6', 'index1_7',
                'index2_1', 'index2_2', 'index2_3', 'index2_4', 'index2_5', 'index2_6', 'index2_7']
        for i in range(1,50):
            string = 'value_'+str(i)
            COLUMN_NAMES.append(string)   
        writer.writerow(COLUMN_NAMES)
        
        ### Read File
        library = parse_liberty(open(liberty_file).read())
        operating_conditions = library.get_groups('operating_conditions')
        print(">>> process :",operating_conditions[0]['process'])
        print(">>> temperature :",operating_conditions[0]['temperature'])
        print(">>> voltage :",operating_conditions[0]['voltage'])
        print("\n\n")

        temperature = operating_conditions[0]['temperature']
        voltage = operating_conditions[0]['voltage']

        cells = library.get_groups('cell')
        cell_list = []
        for cell in cells:
            cell_list.append(cell.args[0])
        print("Cell List :",cell_list)
        for cellname in cell_list:
            # temperature, voltage, capacitance, rise_capacitance, fall_capacitance,
            # cell_rise, rise_transition, cell_fall, fall_transition, rise_power, fall_power,
            # index1 , index2
            # value
            feature = []
            pin_input = []
            cr_index1 = []
            cr_index2 = []
            cr_label = []
            rt_index1 = []
            rt_index2 = []
            rt_label = []
            cf_index1 = []
            cf_index2 = []
            cf_label = []
            ft_index1 = []
            ft_index2 = []
            ft_label = []
            rp_index1 = []
            rp_index2 = []
            rp_label = []
            fp_index1 = []
            fp_index2 = []
            fp_label = []

            rbcf_index1 = []
            rbcf_index2 = []
            rbcf_label = []

            rbft_index1 = []
            rbft_index2 = []
            rbft_label = []

            sbcr_index1 = []
            sbcr_index2 = []
            sbcr_label = []

            sbrt_index1 = []
            sbrt_index2 = []
            sbrt_label = []



            cell = library.get_group('cell', cellname)
            print("Cell :{} ".format(cellname) )
            #writer.writerows(cellname)
            #print(cell)
            pins = cell.get_groups('pin')
            for pin in pins:
                if pin.attributes['direction'] == 'input':
                    print("Pin: ",pin.args)
                    #writer.writerows(pin.args)
                    print(">>> direction:",pin.attributes['direction'])
                    print(">>> capacitance:",pin.attributes['capacitance'])
                    print(">>> rise_capacitance:",pin.attributes['rise_capacitance'])
                    print(">>> fall_capacitance:",pin.attributes['fall_capacitance'])
                    print("\n")
                    pin_input += [[pin.args,pin.attributes['capacitance'],pin.attributes['rise_capacitance'],pin.attributes['fall_capacitance']]]
                elif pin.attributes['direction'] == 'output':
                    print("Pin: ",pin.args)
                    print(">>> direction:",pin.attributes['direction'])
                    print(">>> function:",pin.attributes['function'])

                    timing = pin.get_groups('timing')
                    for timing_c in timing:
                        if timing_c.attributes['related_pin'] == 'RB':
                            #print("related_pin: ",timing_c.attributes['related_pin'])
                            cell_fall_index1 = timing_c.get_group('cell_fall').get_array('index_1')
                            cell_fall_index2 = timing_c.get_group('cell_fall').get_array('index_2')
                            cell_fall_value = timing_c.get_group('cell_fall').get_array('values')
                            #print(">>> cell_fall")
                            #print(">>> cell_fall_index1\n",cell_fall_index1)
                            #print(">>> cell_fall_index2\n",cell_fall_index2)
                            #print(">>> cell_fall_value\n",cell_fall_value)
                            rbcf_index1 += [cell_fall_index1]
                            rbcf_index2 += [cell_fall_index2]
                            rbcf_label += [cell_fall_value]

                            fall_transition_index1 = timing_c.get_group('fall_transition').get_array('index_1')
                            fall_transition_index2 = timing_c.get_group('fall_transition').get_array('index_2')
                            fall_transition_value = timing_c.get_group('fall_transition').get_array('values')
                            #print(">>> fall_transition")
                            #print(">>> fall_transition_index1\n",fall_transition_index1)
                            #print(">>> fall_transition_index2\n",fall_transition_index2)
                            #print(">>> fall_transition_value\n",fall_transition_value)
                            rbft_index1 += [fall_transition_index1]
                            rbft_index2 += [fall_transition_index2]
                            rbft_label += [fall_transition_value]
                        elif timing_c.attributes['related_pin'] == 'SB':
                            #print("related_pin: ",timing_c.attributes['related_pin'])
                            cell_rise_index1 = timing_c.get_group('cell_rise').get_array('index_1')
                            cell_rise_index2 = timing_c.get_group('cell_rise').get_array('index_2')
                            cell_rise_value = timing_c.get_group('cell_rise').get_array('values')
                            #print(">>> array_cell_rise")
                            #print(">>> array_cell_rise_index1\n",cell_rise_index1)
                            #print(">>> array_cell_rise_index2\n",cell_rise_index2)
                            #print(">>> array_cell_rise_value\n",cell_rise_value)
                            sbcr_index1 += [cell_rise_index1]
                            sbcr_index2 += [cell_rise_index2]
                            sbcr_label += [cell_rise_value]


                            rise_transition_index1 = timing_c.get_group('rise_transition').get_array('index_1')
                            rise_transition_index2 = timing_c.get_group('rise_transition').get_array('index_2')
                            rise_transition_value = timing_c.get_group('rise_transition').get_array('values')
                            #print(">>> rise_transition")
                            #print(">>> rise_transition_index1\n",rise_transition_index1)
                            #print(">>> rise_transition_index2\n",rise_transition_index2)
                            #print(">>> rise_transition_value\n",rise_transition_value)
                            sbrt_index1 += [rise_transition_index1]
                            sbrt_index2 += [rise_transition_index2]
                            sbrt_label += [rise_transition_value]
                        else : 

                            #print("related_pin: ",timing_c.attributes['related_pin'])
                            cell_rise_index1 = timing_c.get_group('cell_rise').get_array('index_1')
                            cell_rise_index2 = timing_c.get_group('cell_rise').get_array('index_2')
                            cell_rise_value = timing_c.get_group('cell_rise').get_array('values')
                            #print(">>> array_cell_rise")
                            #print(">>> array_cell_rise_index1\n",cell_rise_index1)
                            #print(">>> array_cell_rise_index2\n",cell_rise_index2)
                            #print(">>> array_cell_rise_value\n",cell_rise_value)
                            cr_index1 += [cell_rise_index1]
                            cr_index2 += [cell_rise_index2]
                            cr_label += [cell_rise_value]

                            rise_transition_index1 = timing_c.get_group('rise_transition').get_array('index_1')
                            rise_transition_index2 = timing_c.get_group('rise_transition').get_array('index_2')
                            rise_transition_value = timing_c.get_group('rise_transition').get_array('values')
                            #print(">>> rise_transition")
                            #print(">>> rise_transition_index1\n",rise_transition_index1)
                            #print(">>> rise_transition_index2\n",rise_transition_index2)
                            #print(">>> rise_transition_value\n",rise_transition_value)
                            rt_index1 += [rise_transition_index1]
                            rt_index2 += [rise_transition_index2]
                            rt_label += [rise_transition_value]

                            cell_fall_index1 = timing_c.get_group('cell_fall').get_array('index_1')
                            cell_fall_index2 = timing_c.get_group('cell_fall').get_array('index_2')
                            cell_fall_value = timing_c.get_group('cell_fall').get_array('values')
                            #print(">>> cell_fall")
                            #print(">>> cell_fall_index1\n",cell_fall_index1)
                            #print(">>> cell_fall_index2\n",cell_fall_index2)
                            #print(">>> cell_fall_value\n",cell_fall_value)
                            cf_index1 += [cell_fall_index1]
                            cf_index2 += [cell_fall_index2]
                            cf_label += [cell_fall_value]

                            fall_transition_index1 = timing_c.get_group('fall_transition').get_array('index_1')
                            fall_transition_index2 = timing_c.get_group('fall_transition').get_array('index_2')
                            fall_transition_value = timing_c.get_group('fall_transition').get_array('values')
                            #print(">>> fall_transition")
                            #print(">>> fall_transition_index1\n",fall_transition_index1)
                            #print(">>> fall_transition_index2\n",fall_transition_index2)
                            #print(">>> fall_transition_value\n",fall_transition_value)
                            ft_index1 += [fall_transition_index1]
                            ft_index2 += [fall_transition_index2]
                            ft_label += [fall_transition_value]



                    internal_power = pin.get_groups('internal_power')
                    for ip in internal_power:
                        #print("related_pin: ",ip.attributes['related_pin'])
                        rise_power_index1 = ip.get_group('rise_power').get_array('index_1')
                        rise_power_index2 = ip.get_group('rise_power').get_array('index_2')
                        rise_power_value = ip.get_group('rise_power').get_array('values')
                        #print(">>> rise_power")
                        #print(">>> rise_power_index1\n",rise_power_index1)
                        #print(">>> rise_power_index2\n",rise_power_index2)
                        #print(">>> rise_power_value\n",rise_power_value)
                        rp_index1 += [[rise_power_index1]]
                        rp_index2 += [[rise_power_index2]]
                        rp_label += [[rise_power_value]]

                        fall_power_index1 = ip.get_group('fall_power').get_array('index_1')
                        fall_power_index2 = ip.get_group('fall_power').get_array('index_2')
                        fall_power_value = ip.get_group('fall_power').get_array('values')
                        #print(">>> fall_power")
                        #print(">>> fall_power_index1\n",fall_power_index1)
                        #print(">>> fall_power_index2\n",fall_power_index2)
                        #print(">>> fall_power_value\n",fall_power_value)
                        fp_index1 += [[fall_power_index1]]
                        fp_index2 += [[fall_power_index2]]
                        fp_label += [[fall_power_value]]

                    print("\n")

            #print(pin_input)
            cr = [1,0,0,0,0,0]
            rt = [0,1,0,0,0,0]
            cf = [0,0,1,0,0,0]
            ft = [0,0,0,1,0,0]
            for i in range(len(timing)):
                for j in range(len(pin_input)):
                    if [timing[i].attributes['related_pin']] == pin_input[j][0]:
                        crf = pin_input[j][1:]
                wr1 = ''
                wr2 = ''
                wr3 = ''
                wr4 = ''
                if timing[i].attributes['related_pin'] == 'RB' :
                    crf = [0.,0.,0.]
                    #print("timing pin:",timing[i].attributes['related_pin'] )
                    #print("cf")
                    #print((list(deepflatten([temperature]+[voltage]+[crf]+[cf]+[rbcf_index1[0]]+[rbcf_index2[0]]+[rbcf_label[0]]))))
                    wr1 = list(deepflatten([temperature]+[voltage]+[crf]+[cf]+[rbcf_index1[0]]+[rbcf_index2[0]]+[rbcf_label[0]]))
                    #print("ft")
                    #print((list(deepflatten([temperature]+[voltage]+[crf]+[ft]+[rbft_index1[0]]+[rbft_index2[0]]+[rbft_label[0]]))))
                    wr2 = list(deepflatten([temperature]+[voltage]+[crf]+[ft]+[rbft_index1[0]]+[rbft_index2[0]]+[rbft_label[0]]))
                elif timing[i].attributes['related_pin'] == 'SB' :
                    crf = [0.,0.,0.]
                    #print("timing pin:",timing[i].attributes['related_pin'] )
                    #print("cr")
                    #print((list(deepflatten([temperature]+[voltage]+[crf]+[cr]+[sbcr_index1[0]]+[sbcr_index2[0]]+[sbcr_label[0]]))))
                    wr1 = list(deepflatten([temperature]+[voltage]+[crf]+[cr]+[sbcr_index1[0]]+[sbcr_index2[0]]+[sbcr_label[0]]))
                    #print("rt")
                    #print((list(deepflatten([temperature]+[voltage]+[crf]+[rt]+[sbrt_index1[0]]+[sbrt_index2[0]]+[sbrt_label[0]]))))
                    wr2 = list(deepflatten([temperature]+[voltage]+[crf]+[rt]+[sbrt_index1[0]]+[sbrt_index2[0]]+[sbrt_label[0]]))
                
                else :  
                    #print("timing pin:",timing[i].attributes['related_pin'] )
                    #print("cr")
                    #print((list(deepflatten([temperature]+[voltage]+[crf]+[cr]+[cr_index1[i]]+[cr_index2[i]]+[cr_label[i]]))))
                    wr1 = list(deepflatten([temperature]+[voltage]+[crf]+[cr]+[cr_index1[i]]+[cr_index2[i]]+[cr_label[i]]))
                    
                    #print("rt")
                    #print((list(deepflatten([temperature]+[voltage]+[crf]+[rt]+[rt_index1[i]]+[rt_index2[i]]+[rt_label[i]]))))
                    wr2 = list(deepflatten([temperature]+[voltage]+[crf]+[rt]+[rt_index1[i]]+[rt_index2[i]]+[rt_label[i]])) 
                    
                    #print("cf")
                    #print((list(deepflatten([temperature]+[voltage]+[crf]+[cf]+[cf_index1[i]]+[cf_index2[i]]+[cf_label[i]]))))
                    wr3 = list(deepflatten([temperature]+[voltage]+[crf]+[cf]+[cf_index1[i]]+[cf_index2[i]]+[cf_label[i]]))
                    
                    #print("ft")
                    #print((list(deepflatten([temperature]+[voltage]+[crf]+[ft]+[ft_index1[i]]+[ft_index2[i]]+[ft_label[i]]))))
                    wr4 = list(deepflatten([temperature]+[voltage]+[crf]+[ft]+[ft_index1[i]]+[ft_index2[i]]+[ft_label[i]]))
                    
                    writer.writerow(wr1)
                    writer.writerow(wr2)
                    writer.writerow(wr3)
                    writer.writerow(wr4)
                    
            for i in range(len(internal_power)):
                for j in range(len(pin_input)):
                    if [internal_power[i].attributes['related_pin']] == pin_input[j][0]:
                        crf = pin_input[j][1:]
                wr5 = ''
                wr6 = ''
                #print("power pin:",internal_power[i].attributes['related_pin'] )
                #print("rp")
                rp = [0,0,0,0,1,0]
                #print((list(deepflatten([temperature]+[voltage]+[crf]+[rp]+[rp_index1[i]]+[rp_index2[i]]+[rp_label[i]]))))
                wr5 = list(deepflatten([temperature]+[voltage]+[crf]+[rp]+[rp_index1[i]]+[rp_index2[i]]+[rp_label[i]]))
                
                #print("fp")
                fp = [0,0,0,0,0,1]
                #print((list(deepflatten([temperature]+[voltage]+[crf]+[fp]+[fp_index1[i]]+[fp_index2[i]]+[fp_label[i]]))))
                wr6 = list(deepflatten([temperature]+[voltage]+[crf]+[fp]+[fp_index1[i]]+[fp_index2[i]]+[fp_label[i]]))
            
                writer.writerow(wr5)
                writer.writerow(wr6)
            #break
            
    #writer.close()
    
if __name__ == '__main__':
    RawDataPath = r"C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\boost\RawData"
    CsvPath = r"C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\boost\csv"
    all_tlib = glob.glob(RawDataPath+'\\*.lib')
    print(all_tlib)
    for i in all_tlib:
        
        file,filesplit = os.path.split(i)
        #print(filesplit.split('.')[:-1])
        OUTPATH = os.path.join(CsvPath,filesplit.split('.')[0])
        OUTPATH = OUTPATH+".csv"
        print(OUTPATH)
        RwaDate2Csv( i,OUTPATH )
    
    
    
    
    
    

['C:\\Users\\User\\Fast_Timing_Model_Estimation_for_new_PVT-master\\test\\boost\\RawData\\lib1_ff1p1v125c_beta_100.lib', 'C:\\Users\\User\\Fast_Timing_Model_Estimation_for_new_PVT-master\\test\\boost\\RawData\\lib1_ff1p1vm40c_beta_100.lib', 'C:\\Users\\User\\Fast_Timing_Model_Estimation_for_new_PVT-master\\test\\boost\\RawData\\lib1_ss0p9v125c_beta_100.lib', 'C:\\Users\\User\\Fast_Timing_Model_Estimation_for_new_PVT-master\\test\\boost\\RawData\\lib1_ss0p9vm40c_beta_100.lib', 'C:\\Users\\User\\Fast_Timing_Model_Estimation_for_new_PVT-master\\test\\boost\\RawData\\lib1_tt1p0v25c_beta_100.lib']
C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\boost\csv\lib1_ff1p1v125c_beta_100.csv
>>> process : 1.0
>>> temperature : 125.0
>>> voltage : 1.1



Cell List : ['AN2AM22', 'AN2M1', 'AN3M1', 'AN4M1', 'AO211M4', 'AO21M2', 'AO221M4', 'AO222M4', 'AO22M2', 'AO31M4', 'AO32M4', 'AO33M4', 'AOI211M4', 'AOI21M1', 'AOI31M2', 'AOI32M2', 'AOI33M4', 'BUFM14', 'CKAN2M2', 'CKBUFM1', 'CKINVM1'



Cell :AOI211M4 
Pin:  ['Z']
>>> direction: "output"
>>> function: "(!((A1&A2)|B|C))"


Pin:  ['A1']
>>> direction: "input"
>>> capacitance: 0.00121563
>>> rise_capacitance: 0.00120483
>>> fall_capacitance: 0.00121563


Pin:  ['A2']
>>> direction: "input"
>>> capacitance: 0.00139665
>>> rise_capacitance: 0.00139061
>>> fall_capacitance: 0.00139665


Pin:  ['B']
>>> direction: "input"
>>> capacitance: 0.00138697
>>> rise_capacitance: 0.00138697
>>> fall_capacitance: 0.00137499


Pin:  ['C']
>>> direction: "input"
>>> capacitance: 0.00118152
>>> rise_capacitance: 0.00118152
>>> fall_capacitance: 0.00115582


Cell :AOI21M1 
Pin:  ['Z']
>>> direction: "output"
>>> function: "(!((A1&A2)|B))"


Pin:  ['A1']
>>> direction: "input"
>>> capacitance: 0.000562148
>>> rise_capacitance: 0.000556475
>>> fall_capacitance: 0.000562148


Pin:  ['A2']
>>> direction: "input"
>>> capacitance: 0.000525089
>>> rise_capacitance: 0.00052303
>>> fall_capacitance: 0.000525089


Pin:  ['B']
>>> direction: "inpu



Pin:  ['CK']
>>> direction: "input"
>>> capacitance: 0.000744554
>>> rise_capacitance: 0.000744554
>>> fall_capacitance: 0.000732178


Pin:  ['D']
>>> direction: "input"
>>> capacitance: 0.00063161
>>> rise_capacitance: 0.00063161
>>> fall_capacitance: 0.000628248


Pin:  ['SB']
>>> direction: "input"
>>> capacitance: 0.00135173
>>> rise_capacitance: 0.00128343
>>> fall_capacitance: 0.00135173


Cell :DFQZRM4 
Pin:  ['Q']
>>> direction: "output"
>>> function: "IQ"


Pin:  ['CK']
>>> direction: "input"
>>> capacitance: 0.000827089
>>> rise_capacitance: 0.000827089
>>> fall_capacitance: 0.000812054


Pin:  ['D']
>>> direction: "input"
>>> capacitance: 0.000666588
>>> rise_capacitance: 0.000661768
>>> fall_capacitance: 0.000666588


Pin:  ['RB']
>>> direction: "input"
>>> capacitance: 0.000660191
>>> rise_capacitance: 0.000659395
>>> fall_capacitance: 0.000660191


Cell :DFQZRSM4 
Pin:  ['Q']
>>> direction: "output"
>>> function: "IQ"


Pin:  ['CK']
>>> direction: "input"
>>> capacitanc

Cell :MXB2M3 
Pin:  ['Z']
>>> direction: "output"
>>> function: "(!((S&B)|((!S)&A)))"


Pin:  ['A']
>>> direction: "input"
>>> capacitance: 0.000708464
>>> rise_capacitance: 0.000708464
>>> fall_capacitance: 0.000703224


Pin:  ['B']
>>> direction: "input"
>>> capacitance: 0.00148115
>>> rise_capacitance: 0.00148115
>>> fall_capacitance: 0.00146288


Pin:  ['S']
>>> direction: "input"
>>> capacitance: 0.0016013
>>> rise_capacitance: 0.0016013
>>> fall_capacitance: 0.00157812


Cell :MXB3M4 
Pin:  ['Z']
>>> direction: "output"
>>> function: "(!((S1&C)|((!S1)&((S0&B)|((!S0)&A)))))"


Pin:  ['A']
>>> direction: "input"
>>> capacitance: 0.000851259
>>> rise_capacitance: 0.000851259
>>> fall_capacitance: 0.000845512


Pin:  ['B']
>>> direction: "input"
>>> capacitance: 0.0010013
>>> rise_capacitance: 0.0010013
>>> fall_capacitance: 0.000995882


Pin:  ['C']
>>> direction: "input"
>>> capacitance: 0.00107076
>>> rise_capacitance: 0.00107076
>>> fall_capacitance: 0.00106464


Pin:  ['S0']
>>>



Pin:  ['A1']
>>> direction: "input"
>>> capacitance: 0.000568328
>>> rise_capacitance: 0.000568328
>>> fall_capacitance: 0.000567401


Pin:  ['A2']
>>> direction: "input"
>>> capacitance: 0.000569797
>>> rise_capacitance: 0.000563023
>>> fall_capacitance: 0.000569797


Pin:  ['B']
>>> direction: "input"
>>> capacitance: 0.000688407
>>> rise_capacitance: 0.000688407
>>> fall_capacitance: 0.0006849


Cell :OA221M4 
Pin:  ['Z']
>>> direction: "output"
>>> function: "((A1|A2)&(B1|B2)&C)"


Pin:  ['A1']
>>> direction: "input"
>>> capacitance: 0.000660548
>>> rise_capacitance: 0.000660548
>>> fall_capacitance: 0.000658529


Pin:  ['A2']
>>> direction: "input"
>>> capacitance: 0.000668039
>>> rise_capacitance: 0.000661973
>>> fall_capacitance: 0.000668039


Pin:  ['B1']
>>> direction: "input"
>>> capacitance: 0.000630278
>>> rise_capacitance: 0.000630127
>>> fall_capacitance: 0.000630278


Pin:  ['B2']
>>> direction: "input"
>>> capacitance: 0.000872979
>>> rise_capacitance: 0.0008656
>>> f

...

>>> process : 1.0
>>> temperature : 25.0
>>> voltage : 1.0



Cell List : ['AN2AM22', 'AN2M1', 'AN3M1', 'AN4M1', 'AO211M4', 'AO21M2', 'AO221M4', 'AO222M4', 'AO22M2', 'AO31M4', 'AO32M4', 'AO33M4', 'AOI211M4', 'AOI21M1', 'AOI31M2', 'AOI32M2', 'AOI33M4', 'BUFM14', 'CKAN2M2', 'CKBUFM1', 'CKINVM1', 'CKMUX2M2', 'CKND2M2', 'CKXOR2M2', 'DFCQM4', 'DFCQRM4', 'DFCQRSM4', 'DFCQSM4', 'DFEQM4', 'DFEQRM4', 'DFEQZRM4', 'DFMQM4', 'DFQM4', 'DFQRM4', 'DFQRSM4', 'DFQSM4', 'DFQZRM4', 'DFQZRSM4', 'DFQZSM4', 'INVM14', 'LACM4', 'LACQM4', 'LAM4', 'LAQM4', 'LAQRM4', 'LAQSM4', 'LARM4', 'LARSM4', 'LASM4', 'MUX2M2', 'MUX3M4', 'MUX4M4', 'MXB2M3', 'MXB3M4', 'MXB4M4', 'ND2B1M1', 'ND2M1', 'ND3B1M2', 'ND3M1', 'ND4B1M4', 'ND4B2M4', 'ND4M2', 'NR2B1M1', 'NR2M1', 'NR3B1M4', 'NR3M2', 'NR4B1M4', 'NR4B2M4', 'NR4M2', 'OA211M2', 'OA21M1', 'OA221M4', 'OA222M4', 'OA22M1', 'OA31M4', 'OA32M4', 'OA33M4', 'OAI21M1', 'OAI31M4', 'OAI32M4', 'OAI33M4', 'OR2M1', 'OR3M1', 'OR4M1', 'OR6M2', 'SDFCQM4', 'SDFCQRM4', 'SDFCQSM4', 'SDFEQM4', 'SDF

>>> function: "(!((A1&A2)|B|C))"


Pin:  ['A1']
>>> direction: "input"
>>> capacitance: 0.00103533
>>> rise_capacitance: 0.00102787
>>> fall_capacitance: 0.00103533


Pin:  ['A2']
>>> direction: "input"
>>> capacitance: 0.00118454
>>> rise_capacitance: 0.00118023
>>> fall_capacitance: 0.00118454


Pin:  ['B']
>>> direction: "input"
>>> capacitance: 0.00118173
>>> rise_capacitance: 0.00118173
>>> fall_capacitance: 0.00116506


Pin:  ['C']
>>> direction: "input"
>>> capacitance: 0.00101859
>>> rise_capacitance: 0.00101859
>>> fall_capacitance: 0.000992273


Cell :AOI21M1 
Pin:  ['Z']
>>> direction: "output"
>>> function: "(!((A1&A2)|B))"


Pin:  ['A1']
>>> direction: "input"
>>> capacitance: 0.000474936
>>> rise_capacitance: 0.000470681
>>> fall_capacitance: 0.000474936


Pin:  ['A2']
>>> direction: "input"
>>> capacitance: 0.000441258
>>> rise_capacitance: 0.000438912
>>> fall_capacitance: 0.000441258


Pin:  ['B']
>>> direction: "input"
>>> capacitance: 0.000511336
>>> rise_capacitance



Pin:  ['CK']
>>> direction: "input"
>>> capacitance: 0.000692697
>>> rise_capacitance: 0.000692697
>>> fall_capacitance: 0.000677612


Pin:  ['D']
>>> direction: "input"
>>> capacitance: 0.000788535
>>> rise_capacitance: 0.000788535
>>> fall_capacitance: 0.000771172


Pin:  ['SB']
>>> direction: "input"
>>> capacitance: 0.000650327
>>> rise_capacitance: 0.000650327
>>> fall_capacitance: 0.00063911


Cell :INVM14 
Pin:  ['Z']
>>> direction: "output"
>>> function: "(!A)"


Pin:  ['A']
>>> direction: "input"
>>> capacitance: 0.00466904
>>> rise_capacitance: 0.00466904
>>> fall_capacitance: 0.00458573


Cell :LACM4 
Pin:  ['Q']
>>> direction: "output"
>>> function: "IQ"


Pin:  ['QB']
>>> direction: "output"
>>> function: "IQN"


Pin:  ['D']
>>> direction: "input"
>>> capacitance: 0.000540712
>>> rise_capacitance: 0.000540712
>>> fall_capacitance: 0.000540036


Pin:  ['GB']
>>> direction: "input"
>>> capacitance: 0.000644538
>>> rise_capacitance: 0.000644538
>>> fall_capacitance: 0.00063

>>> capacitance: 0.00125401
>>> rise_capacitance: 0.00125401
>>> fall_capacitance: 0.00124474


Pin:  ['C']
>>> direction: "input"
>>> capacitance: 0.000935096
>>> rise_capacitance: 0.000935096
>>> fall_capacitance: 0.000926751


Pin:  ['D']
>>> direction: "input"
>>> capacitance: 0.00122684
>>> rise_capacitance: 0.00122684
>>> fall_capacitance: 0.00120315


Pin:  ['S0']
>>> direction: "input"
>>> capacitance: 0.00232304
>>> rise_capacitance: 0.00232304
>>> fall_capacitance: 0.0022813


Pin:  ['S1']
>>> direction: "input"
>>> capacitance: 0.00142856
>>> rise_capacitance: 0.00142856
>>> fall_capacitance: 0.00140725


Cell :ND2B1M1 
Pin:  ['Z']
>>> direction: "output"
>>> function: "(!((!NA)&B))"


Pin:  ['B']
>>> direction: "input"
>>> capacitance: 0.000480084
>>> rise_capacitance: 0.000480084
>>> fall_capacitance: 0.000476872


Pin:  ['NA']
>>> direction: "input"
>>> capacitance: 0.000582284
>>> rise_capacitance: 0.000582284
>>> fall_capacitance: 0.000572927


Cell :ND2M1 
Pin:  ['Z']


Cell :OA22M1 
Pin:  ['Z']
>>> direction: "output"
>>> function: "((A1|A2)&(B1|B2))"


Pin:  ['A1']
>>> direction: "input"
>>> capacitance: 0.000551982
>>> rise_capacitance: 0.000551982
>>> fall_capacitance: 0.000550816


Pin:  ['A2']
>>> direction: "input"
>>> capacitance: 0.000531728
>>> rise_capacitance: 0.000531728
>>> fall_capacitance: 0.000526323


Pin:  ['B1']
>>> direction: "input"
>>> capacitance: 0.000514208
>>> rise_capacitance: 0.000514208
>>> fall_capacitance: 0.000508496


Pin:  ['B2']
>>> direction: "input"
>>> capacitance: 0.000595556
>>> rise_capacitance: 0.000595556
>>> fall_capacitance: 0.000583407


Cell :OA31M4 
Pin:  ['Z']
>>> direction: "output"
>>> function: "((A1|A2|A3)&B)"


Pin:  ['A1']
>>> direction: "input"
>>> capacitance: 0.00059189
>>> rise_capacitance: 0.000589474
>>> fall_capacitance: 0.00059189


Pin:  ['A2']
>>> direction: "input"
>>> capacitance: 0.000525461
>>> rise_capacitance: 0.000525461
>>> fall_capacitance: 0.000518776


Pin:  ['A3']
>>> direct

In [46]:

print('1111')

1111
